- 학번 : 201611125
- 이름 : 김정수
- 학과 : 게임전공

## 문제
- 아래 데이터를 RDD로 만들고, 성적의 합계 및 평균을 계산하세요.


이름      |      과목     |      점수
---------|---------|---------
김하나 | English | 100
김하나 | Math | 80
임하나 | English | 70
임하나 | Math | 100
김갑돌 | English | 82.3
김갑돌 | Math | 98.5

In [1]:
import pyspark
import os

In [2]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf = myConf)\
        .getOrCreate()

21/09/29 16:40:05 WARN Utils: Your hostname, jeongxoo.local resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
21/09/29 16:40:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/29 16:40:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = [
    ["김하나", "English", 100],
    ["김하나", "Math", 80],
    ["임하나", "English", 70],
    ["임하나", "Math", 100],
    ["김갑돌", "English", 82.3],
    ["김갑돌", "Math", 98.5]
]

In [4]:
myRDD = spark.sparkContext.parallelize(data)

In [5]:
myRDD.collect()

[['김하나', 'English', 100],
 ['김하나', 'Math', 80],
 ['임하나', 'English', 70],
 ['임하나', 'Math', 100],
 ['김갑돌', 'English', 82.3],
 ['김갑돌', 'Math', 98.5]]

### 문제 3-1: 이름으로 합계를 구해보자. 

* 정답 예시 :
```
'임하나' 170.0
'김하나' 180.0
'김갑돌' 180.8
```
- map()을 사용해 튜플 형태 (이름, 점수) 생성
- key(이름) 기준으로 점수 값의 합 생성

In [6]:
q3_1 = myRDD.map(lambda x: (x[0], x[2]))\
            .reduceByKey(lambda x,y: x + y)\
            .collect()

/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [7]:
for val in q3_1: print("'%s' %.1f" %(val[0], val[1]))

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


### 문제 3-2: 과목으로 합계를 계산해 보자. 


* 정답 예시 :
```
'English' 252.3
'Math' 278.5
```


In [8]:
q3_2 = myRDD.map(lambda x: (x[1], x[2]))\
            .reduceByKey(lambda x,y: x + y)\
            .collect()

In [9]:
for val in q3_2: print("'%s' %.1f" %(val[0], val[1]))

'English' 252.3
'Math' 278.5





### 문제 3-3: 이름으로 합계과 개수를 구해보자. 


* 정답 예시 :
```
'임하나' (170.0, 2)
'김하나' (180.0, 2)
'김갑돌' (180.8, 2)
```

#### combineByKey() 사용 X

In [10]:
q3_3 = myRDD.map(lambda x: (x[0], (x[2], 1)))\
            .reduceByKey(lambda x,y: [x, y])\
            .map(lambda x: [x[0], sum([a[0] for a in x[1]]), sum([a[1] for a in x[1]])])

In [11]:
for val in q3_3.collect(): print("'%s' (%.1f %d)" %(val[0], val[1], val[2]))

'김하나' (180.0 2)
'임하나' (170.0 2)
'김갑돌' (180.8 2)


#### combineByKey() 사용 O

In [12]:
q3_3c = myRDD.map(lambda x: (x[0], x[2]))\
            .combineByKey(
                lambda value: (value, 1), # ex -> key = "김하나", value -> (100, 1)
                lambda x,value: (x[0] + value, x[1] + 1), # 점수 + 점수, count + count
                lambda x,y: (x[0] + y[0], x[1] + y[1])) # partition별 합산

In [13]:
for val in q3_3c.collect(): print("'%s' (%.1f %d)" %(val[0], val[1][0], val[1][1]))

'김하나' (180.0 2)
'임하나' (170.0 2)
'김갑돌' (180.8 2)





### 문제 3-4: 이름으로 평균을 계산해 보자. 앞서 3-3에서 사용했던 결과를 활용하라.




* 정답 예시 :
```
'임하나' 85.0
'김하나' 90.0
'김갑돌' 90.4
```



In [14]:
q3_4 = q3_3.map(lambda x: [x[0], x[1]/x[2]])\
            .collect()

In [15]:
for val in q3_4: print("'%s' %.1f" %(val[0], val[1]))

'김하나' 90.0
'임하나' 85.0
'김갑돌' 90.4
